In [1]:
import csv
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from flask import Flask, request, jsonify
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
# Load Trained ML Model
model = keras.models.load_model('model')

2021-11-21 06:01:37.923821: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load Tokenizer
tokenizer = Tokenizer(num_words=500, oov_token="<OOV>")
sentences = []
with open("dataset/spam_dataset.csv", encoding="ISO-8859-1") as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        sentences.append(row[1])
split = int(len(sentences)*0.85)
training_x = sentences[:split]
tokenizer.fit_on_texts(training_x)

In [ ]:
# Production ML API 

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def hello():
    return "Hello World!"

@app.route("/query/", methods = ['POST'])
def query():
    content = request.form.get("query")
    content = content[0:189]
    padded = pad_sequences(tokenizer.texts_to_sequences([content]), maxlen=189)
    prediction = model.predict(padded)
    confidence = prediction[0][0] * 100
    if confidence < 0.03:
        answer = "real"
    else:
        answer = "fake"
    return { "prediction": answer, "confidence": confidence}

if __name__ == '__main__':
    app.run()